In [ ]:
# set up
import os
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np

os.chdir("../data/")

In [ ]:
# load data
parcels = geopandas.read_file('Parcels_FY19')

In [ ]:
parcels.head()

In [ ]:
# parcel map
fig, ax = plt.subplots(figsize=(20,10))
parcels.plot(ax=ax, color='white', edgecolor='grey', alpha=0.3)
plt.title("All parcels in Somerville", fontsize=22)
plt.axis('off');

In [ ]:
# read in labels
manual_labels = pd.read_csv('../labels/training_labels_updated_111219.csv')
additional_manual_labels = pd.read_csv('../labels/additional_training_labels_120319.csv')
labels = pd.concat([manual_labels, additional_manual_labels], axis=0)

In [ ]:
manual_labels.shape, additional_manual_labels.shape, labels.shape

In [ ]:
labels.head()

In [ ]:
labels.final_label.value_counts()

In [ ]:
# join
parcel_labels = parcels.merge(labels, how='left', left_on='TaxParMBL', right_on='MBL')

In [ ]:
parcel_labels.shape

In [ ]:
# denote ones we have labeled
parcel_labels['label_flag'] = pd.notna(parcel_labels['final_label'])

In [ ]:
# map where we labeled
fig, ax = plt.subplots(figsize=(20,10))
parcel_labels.plot(ax=ax, color='white', edgecolor='grey', alpha=0.1)
parcel_labels.plot(ax=ax, column='label_flag', alpha=0.9, legend=True, cmap='OrRd')
plt.title("Labeled parcels (1 - yes, 0 - no)", fontsize=22)
plt.axis('off');

In [ ]:
# map what labels were
label_mapping = {0: 'no', 0.1:'no', 0.5:'unsure', 0.9:'yes', 1:'yes', np.nan:'no label'}
parcel_labels['3_labels'] = parcel_labels['final_label'].map(label_mapping)

In [ ]:
import matplotlib as mpl
cmap = mpl.colors.ListedColormap(["purple", "white", "darkorange", "green"])
fig, ax = plt.subplots(figsize=(20,10))
parcel_labels.plot(ax=ax, color='white', edgecolor='grey', alpha=0.2)
parcel_labels.plot(ax=ax, column='3_labels', alpha=0.9, legend=True, cmap=cmap)
plt.title("", fontsize=22)
plt.axis('off');

# Load predictions

In [ ]:
calibrated_preds = pd.read_csv('../data/calibrated_driveway_predictions.csv')

In [ ]:
calibrated_preds.head()

# Aggregate by block

In [ ]:
calibrated_preds['block'] = calibrated_preds['MBL'].apply(lambda x: x.split('-')[0] + x.split('-')[1])

In [ ]:
block_pred_counts = calibrated_preds.groupby('block').mean()['calibrated_yes_driveway'].to_frame().reset_index()

In [ ]:
block_pred_counts.columns = ['block', 'block_pred_counts']

In [ ]:
calibrated_preds = calibrated_preds.merge(block_pred_counts, how='left', left_on='block', right_on='block')

In [ ]:
parcel_calibrated_preds = parcels.merge(calibrated_preds, how='left', left_on='MBL', right_on='MBL')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,10))
bins = [0.1,0.5,0.6,0.7,0.8,0.9,1]

parcel_calibrated_preds.plot(ax=ax, color='white', edgecolor='white', alpha=0.1)
#parcel_calibrated_preds.plot(ax=ax, column='block_pred_counts', cmap='Reds', alpha=0.7)
parcel_calibrated_preds[parcel_calibrated_preds['block_pred_counts'].notnull()].plot(ax=ax, 
                                                                                  column='block_pred_counts', 
                                                                                  cmap='Reds', 
                                                                                  scheme='user_defined',
                                                                                  classification_kwds={'bins':bins},
                                                                                  legend=False)
# DISCRETE
cmap = plt.cm.Reds 
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
norm = mpl.colors.BoundaryNorm(bins, cmap.N)

# create a second axes for the colorbar
ax_cm = fig.add_axes([0.3, 0.05, 0.4, 0.03]) #left bottom width height
cb = mpl.colorbar.ColorbarBase(ax_cm, cmap=cmap, norm=norm, orientation='horizontal',
    spacing='uniform', ticks=bins, boundaries=bins)



# CONTINUOUS
# # Create colorbar as a legend
# sm = plt.cm.ScalarMappable(cmap='Reds', 
#                            norm=plt.Normalize(vmin=0, vmax=parcel_calibrated_preds['block_pred_counts'].max()))
# # empty array for the data range
# sm._A = []
# # add the colorbar to the figure
# cbar = fig.colorbar(sm, shrink=0.5, orientation='horizontal', pad=0)

ax.axis('off');
#plt.savefig('../images/block_predictions_map', dpi=450);

# Add streets

In [ ]:
streets = geopandas.read_file('Streets')

In [ ]:
streets.head()

In [ ]:
streets['coords'] = streets['geometry'].apply(lambda x: x.representative_point().coords[:])
streets['coords'] = [coords[0] for coords in streets['coords']]
streets['x'] = streets['coords'].apply(lambda x: x[0])
streets['y'] = streets['coords'].apply(lambda x: x[1])

In [ ]:
for i in pd.Series(streets['Street'].unique()).sort_values():
    print(i)

In [ ]:
major = ['BROADWAY', 'HIGHLAND AVE', 'POWDER HOUSE BLVD', 'SUMMER ST', 'WASHINGTON ST']
major_streets = streets[streets['Street'].isin(major)]
major_streets = major_streets.groupby('Street').mean()[['x','y']].reset_index()
major_streets['coords'] = major_streets[['x','y']].apply(tuple, axis=1)

In [ ]:
major_streets

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
streets.plot(ax=ax, alpha=0.5)

for idx, row in major_streets.iterrows():
    plt.annotate(s=row['Street'], xy=row['coords']
                 #,horizontalalignment='center'
                )
    
ax.axis('off');